In [ ]:
import torch
from torch.utils.data import DataLoader, TensorDataset

# 创建一个简单的数据集
data = torch.randn(100, 3)  # 假设有100个样本，每个样本有3个特征
dataset = TensorDataset(data)

# 创建一个 DataLoader，设置 shuffle=False 以保持数据的原始顺序
dataloader = DataLoader(dataset, batch_size=8, shuffle=False)

for k in range(100):
    # 从 DataLoader 中按顺序获取 batch
    for i, batch in enumerate(dataloader):
        # 这里的 batch 将按照索引顺序被加载
        # i 为 0 时，你将得到第一个 batch，即索引为 0 到 7 的元素
        # i 为 1 时，你将得到第二个 batch，即索引为 8 到 15 的元素
        # 以此类推
        indexed_batch = batch[0]
        print(f'Batch {i}: {indexed_batch}')


In [7]:
import torch
from torch.utils.data import DataLoader, Dataset
import random

class Splited_FMRI_dataset(Dataset):
    def __init__(self,):
        self.inputs = list(range(1000))
        self.most_epoch = 5
    def __len__(self):
        if self.most_epoch > -1:
            return min(self.most_epoch, len(self.inputs))
        return len(self.inputs)
    def __getitem__(self, idx):
        input_sample = self.inputs[idx]
        return input_sample
dataset = Splited_FMRI_dataset()

for k in range(100):
    random.shuffle(dataset.inputs)
    dataloader = DataLoader(dataset, batch_size=3, shuffle=True)
    for i, batch in enumerate(dataloader):
        print(f'{k}, Batch {i}: {batch}')

0, Batch 0: tensor([2, 0, 1])
0, Batch 1: tensor([3, 4])
1, Batch 0: tensor([2, 1, 3])
1, Batch 1: tensor([4, 0])
2, Batch 0: tensor([1, 3, 2])
2, Batch 1: tensor([0, 4])
3, Batch 0: tensor([3, 4, 1])
3, Batch 1: tensor([2, 0])
4, Batch 0: tensor([4, 3, 0])
4, Batch 1: tensor([2, 1])
5, Batch 0: tensor([1, 4, 0])
5, Batch 1: tensor([2, 3])
6, Batch 0: tensor([2, 3, 4])
6, Batch 1: tensor([0, 1])
7, Batch 0: tensor([3, 0, 4])
7, Batch 1: tensor([2, 1])
8, Batch 0: tensor([3, 0, 4])
8, Batch 1: tensor([2, 1])
9, Batch 0: tensor([3, 4, 0])
9, Batch 1: tensor([2, 1])
10, Batch 0: tensor([4, 0, 2])
10, Batch 1: tensor([1, 3])
11, Batch 0: tensor([0, 1, 4])
11, Batch 1: tensor([3, 2])
12, Batch 0: tensor([1, 0, 2])
12, Batch 1: tensor([3, 4])
13, Batch 0: tensor([3, 2, 4])
13, Batch 1: tensor([1, 0])
14, Batch 0: tensor([3, 1, 2])
14, Batch 1: tensor([0, 4])
15, Batch 0: tensor([0, 2, 1])
15, Batch 1: tensor([3, 4])
16, Batch 0: tensor([3, 1, 4])
16, Batch 1: tensor([0, 2])
17, Batch 0: tens

In [8]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AutoTokenizer, AutoModel, LlamaForCausalLM, LlamaTokenizer
tokenizer = GPT2Tokenizer.from_pretrained('/home/yzy/.cache/huggingface/hub/models--gpt2-large/snapshots/97935fc1a406f447320c3db70fe9e9875dca2595')
model = GPT2LMHeadModel.from_pretrained('/home/yzy/.cache/huggingface/hub/models--gpt2-large/snapshots/97935fc1a406f447320c3db70fe9e9875dca2595')





In [134]:
tokenizer.pad_token = tokenizer.eos_token
content_true1 = 'Beijing is'
content_true1 = 'The weather today is'
input_ids1 = tokenizer.encode_plus(content_true1, max_length=32, truncation=False, return_tensors='pt',padding='max_length')

content_prev1 = model.transformer.wte(input_ids1['input_ids'])

padding_counts = (input_ids1['attention_mask'] == 0).sum(dim=1)

# 初始化新的张量
front_padded_input_embeds = torch.zeros_like(content_prev1)
front_padded_mask = torch.zeros_like(input_ids1['attention_mask'])

for i in range(content_prev1.size(0)):  # 遍历每个样本
    # 计算需要移动的位置数
    shift = padding_counts[i].item()
    
    # 为 input_embeds 和 mask 重新排列
    front_padded_input_embeds[i, content_prev1.size(1) - shift:] = content_prev1[i, :shift]
    front_padded_mask[i, content_prev1.size(1) - shift:] = input_ids1['attention_mask'][i, :shift]

content_prev1 = front_padded_input_embeds
input_ids1['attention_mask'] = front_padded_mask

input_ids2 = tokenizer.encode_plus(content_true2, max_length=32, truncation=False, return_tensors='pt',padding='max_length')


# content_prev2 = model.transformer.wte(input_ids2['input_ids'])
content_prev = torch.cat([content_prev1, ], dim=0)
# model.generate(inputs_embeds = content_prev, min_length = len(content_prev[0])+4, max_length=len(content_prev[0])+32,return_dict_in_generate=True,num_beams=1,do_sample=False,attention_mask=torch.ones(content_prev.shape[:-1], dtype=torch.long, device=self.device), pad_token_id=self.tokenizer.eos_token_id)
content_prev_mask = torch.cat([input_ids1['attention_mask'], ], dim=0)

# re1 = model.generate(inputs_embeds = content_prev, attention_mask = content_prev_mask,return_dict_in_generate=True,min_length = 10, max_length=32,num_beams=1,do_sample=False, repetition_penalty=2.0, pad_token_id=tokenizer.eos_token_id)
re1 = model.generate(inputs_embeds = content_prev, attention_mask = content_prev_mask,return_dict_in_generate=True,min_length = 10, max_length=32,do_sample=False, repetition_penalty=2.0, pad_token_id=tokenizer.eos_token_id)

re = [tokenizer.convert_ids_to_tokens(re1['sequences'][0]), ]
print(tokenizer.convert_tokens_to_string(re[0]))












<|endoftext|> the best I've seen in a long time. It's been so hot and humid that it feels like we're living on Mars."

The weather


In [132]:
content_prev1.size(1)

32

In [118]:
input_ids1 = tokenizer.encode_plus(content_true1, max_length=32, truncation=False, return_tensors='pt',padding='max_length')
input_ids1['input_ids'] = torch.cat([torch.tensor([tokenizer.pad_token_id] * padding_length), input_ids1['input_ids'][0]]).long()
input_ids1['attention_mask'] = torch.cat([torch.tensor([0] * padding_length), input_ids1['attention_mask'][0]])

In [130]:
input_ids1['attention_mask']

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])

In [124]:
torch.cat([torch.tensor([0] * padding_length), input_ids1['attention_mask'][0]]).unsqueeze(0)

tensor([[1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [109]:
content_prev.shape

torch.Size([32, 1280])

In [110]:
content_prev_mask

tensor([1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [98]:
a=model.generate(inputs_embeds = content_prev, attention_mask = content_prev_mask, min_length = 10, max_length=32,return_dict_in_generate=True,num_beams=5,do_sample=False, repetition_penalty=2.0,pad_token_id=tokenizer.eos_token_id)
content_prev = content_prev[content_prev_mask==1].unsqueeze(0)
b=model.generate(inputs_embeds =  content_prev, attention_mask = torch.ones(content_prev.shape[:-1], dtype=torch.long,), min_length = 10, max_length=32,return_dict_in_generate=True,num_beams=5,do_sample=False, repetition_penalty=2.0,pad_token_id=tokenizer.eos_token_id)





In [99]:
print(a)
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(a['sequences'][0])))

BeamSearchDecoderOnlyOutput(sequences=tensor([[50256,   464,   995,   338,   749, 44042,  1748,    11,   351,   257,
          3265,   286,   517,   621,   352,    13,    18,  2997,   661,    13,
           632,   318,   635,  1363,   284,   262,   995,   338,  1218,    12,
         28209,  3773]]), sequences_scores=None, scores=None, beam_indices=None, attentions=None, hidden_states=None)
<|endoftext|>The world's most populous city, with a population of more than 1.3 billion people. It is also home to the world's second-largest economy


In [100]:
print(b)
print(tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(b['sequences'][0])))

BeamSearchDecoderOnlyOutput(sequences=tensor([[50256,  1849,  1169,   691,  1499,   287,   262,   995,   326,   468,
           407, 39067,   262, 11680,   319,   262, 27405,  1883,   286,  1439,
         39196,   286, 49298,  1028,  6926,   357,    34,  1961, 12298,   737,
           220,  1849]]), sequences_scores=None, scores=None, beam_indices=None, attentions=None, hidden_states=None)
<|endoftext|> the only country in the world that has not ratified the Convention on the Elimination of All Forms of Discrimination against Women (CEDAW).  


In [53]:
content_prev.shape

torch.Size([2, 32, 1280])

In [44]:
len(re1['sequences'])

2

In [37]:
help(model.generate)

Help on method generate in module transformers.generation.utils:

generate(inputs: Union[torch.Tensor, NoneType] = None, generation_config: Union[transformers.generation.configuration_utils.GenerationConfig, NoneType] = None, logits_processor: Union[transformers.generation.logits_process.LogitsProcessorList, NoneType] = None, stopping_criteria: Union[transformers.generation.stopping_criteria.StoppingCriteriaList, NoneType] = None, prefix_allowed_tokens_fn: Union[Callable[[int, torch.Tensor], List[int]], NoneType] = None, synced_gpus: Union[bool, NoneType] = None, assistant_model: Union[ForwardRef('PreTrainedModel'), NoneType] = None, streamer: Union[ForwardRef('BaseStreamer'), NoneType] = None, **kwargs) -> Union[transformers.generation.utils.GreedySearchEncoderDecoderOutput, transformers.generation.utils.GreedySearchDecoderOnlyOutput, transformers.generation.utils.SampleEncoderDecoderOutput, transformers.generation.utils.SampleDecoderOnlyOutput, transformers.generation.utils.BeamSearc

In [33]:
tokenizer.convert_ids_to_tokens(re1)

ValueError: only one element tensors can be converted to Python scalars

In [28]:
content_prev.shape

torch.Size([2, 32, 1280])

In [15]:
input_ids.keys()

dict_keys(['input_ids', 'attention_mask'])

In [16]:
input_ids['attention_mask']

tensor([[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]])